In [4]:
import pandas as pd
import numpy as np

In [5]:
import re

In [6]:
links = pd.read_csv("../data/ml-100k/ml-latest-small/links.csv")
movies = pd.read_csv("../data/ml-100k/ml-latest-small/movies.csv")
ratings = pd.read_csv("../data/ml-100k/ml-latest-small/ratings.csv")
tags = pd.read_csv("../data/ml-100k/ml-latest-small/tags.csv")

** link.csv 는 사용안하는듯 **

In [7]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [11]:
# load_ratings_df

df = pd.read_csv("../data/ml-100k/ml-latest-small/ratings.csv")
df.columns = ['uid', 'sid', 'rating', 'timestamp']
df

,uid,sid,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [12]:
# load_meta_dict (dataframe -> dictionary)

movies_df = pd.read_csv("../data/ml-100k/ml-latest-small/movies.csv")
meta_dict = {}
for row in movies_df.itertuples():
    title = row[2][:-7] # remove year (optional)
    year = row[2][-7:]

    title = re.sub('\(.*?\)', '', title).strip()

    if any(', ' + x in title.lower()[-5:] for x in ['a', 'an', 'the']):
        title_pre = title.split(', ')[:-1]
        title_post = title.split(', ')[-1]
        title_pre = ', '.join(title_pre)

        title = title_post + ' ' + title_pre

    meta_dict[row[1]] = title + year

meta_dict

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'The American President (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'The City of Lost Children (1995)',
 30: 'Shanghai Triad (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 34: 'Babe (1995)',
 36: 'Dead Man Walking (1995)',
 38: 'It Takes Two (1995)'

In [13]:
df = df[df['sid'].isin(meta_dict)]

In [14]:
df

,uid,sid,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [15]:
# densify_index

umap = {u: i for i, u in enumerate(set(df['uid']), start=1)}
smap = {s: i for i, s in enumerate(set(df['sid']), start=1)}

df['uid'] = df['uid'].map(umap)
df['sid'] = df['sid'].map(smap)

df

,uid,sid,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,2292,4.0,1493848402
100832,610,3715,5.0,1493850091
100833,610,3717,5.0,1494273047
100834,610,3719,5.0,1493846352


In [24]:
# split_df

user_count = len(umap)
user_group = df.groupby('uid')
user2items = user_group.apply(
    lambda d: list(d.sort_values(by=['timestamp', 'sid'])['sid']))

train, val, test = {}, {}, {}
for i in range(user_count):
    user = i + 1
    items = user2items[user]
    # train은 첫번째 ~ 맨뒤 + 2 사이의 값들 (list)
    # val은 맨뒤에서 2번째 값
    # test은 맨뒤 값
    train[user], val[user], test[user] = items[:-2], items[-2:-1], items[-1:]

C:\Users\sinny\AppData\Local\Temp\ipykernel_12628\1488635950.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user2items = user_group.apply(


In [40]:
meta = {smap[k]: v for k, v in meta_dict.items() if k in smap}
meta

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'The American President (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'The City of Lost Children (1995)',
 35: 'Shanghai Triad (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 34: 'Babe (1995)',
 36: 'Dead Man Walking (1995)',
 38: 'It Takes Two (1995)'

In [41]:
dataset = {
    'train': train,
    'val': val,
    'test': test,
    'meta': meta,
    'umap': umap,
    'smap': smap
}

In [42]:
dataset

{'train': {1: [710,
   1033,
   1705,
   2241,
   2416,
   3033,
   3062,
   3176,
   99,
   423,
   2437,
   1269,
   2563,
   223,
   921,
   342,
   212,
   1289,
   2297,
   2039,
   2765,
   1303,
   2774,
   1345,
   1480,
   2213,
   2931,
   219,
   319,
   519,
   905,
   205,
   478,
   2612,
   3236,
   1514,
   3,
   2167,
   2132,
   935,
   983,
   1522,
   1079,
   1616,
   805,
   3103,
   248,
   931,
   952,
   1853,
   2451,
   353,
   1654,
   1692,
   1774,
   1841,
   1872,
   2388,
   1786,
   1824,
   2956,
   1738,
   616,
   872,
   1909,
   2961,
   1609,
   2920,
   2921,
   1019,
   1021,
   1037,
   2289,
   3223,
   1020,
   2513,
   2622,
   1715,
   2192,
   2518,
   439,
   1043,
   1045,
   1114,
   3135,
   108,
   821,
   1641,
   2516,
   2517,
   1687,
   6,
   2071,
   2155,
   2559,
   557,
   2557,
   2991,
   3081,
   1098,
   1343,
   2489,
   302,
   1219,
   1929,
   2050,
   2154,
   2922,
   459,
   1352,
   405,
   3161,
   2248,
   70,
